# Laboratory work #6 (QA)

In [1]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

from db_utils import ChromaDataBase

In [2]:
model_name = "TheBloke/Mistral-7B-OpenOrca-GGUF"
model_file = "mistral-7b-openorca.Q4_K_M.gguf"

In [3]:
model_path = hf_hub_download(model_name, filename=model_file)

In [4]:
model_kwargs = {
  "n_ctx":4096,    # Context length to use
  "n_threads":4,   # Number of CPU threads to use
  "n_gpu_layers":0,# Number of model layers to offload to GPU. Set to 0 if only using CPU
}

## Instantiate model from downloaded file
llm = Llama(model_path=model_path, **model_kwargs)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /home/azureuser/.cache/huggingface/hub/models--TheBloke--Mistral-7B-OpenOrca-GGUF/snapshots/fbd9cd059e5fa0bba72a0abe8aea7e127d7994cb/mistral-7b-openorca.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = open-orca_mistral-7b-openorca
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:

In [5]:
## Generation kwargs
generation_kwargs = {
    "max_tokens":200, # Max number of new tokens to generate
    "stop":["<|endoftext|>", "</s>"], # Text sequences to stop generation on
    "echo":False, # Echo the prompt in the output
    "top_k":1 # This is essentially greedy decoding, since the model will always return the highest-probability token. Set this value > 1 for sampling decoding
}

prompt = "The meaning of life is "

In [6]:
res = llm(prompt, **generation_kwargs) # Res is a dictionary

## Unpack and the generated text from the LLM response dictionary and print it
print(res["choices"][0]["text"])

42, according to the Hitchhiker's Guide to the Galaxy. But what if you're not a fan of that particular science fiction series? What if you're looking for something more grounded in reality or philosophy?

Well, there are many different theories and ideas about the meaning of life. Some people believe it is to find happiness, others think it is to serve a greater purpose, while some might argue that there is no inherent meaning at all. Here are ten different perspectives on the meaning of life:

1. The religious perspective: Many religions propose that the meaning of life is to worship God or a higher power and follow their teachings in order to achieve spiritual growth and salvation.

2. The existentialist perspective: Existentialists believe that humans create their own meaning in life through individual choices, experiences, and relationships. They argue that there is no predetermined purpose for existence.

3



llama_print_timings:        load time =     183.69 ms
llama_print_timings:      sample time =      31.46 ms /   200 runs   (    0.16 ms per token,  6356.87 tokens per second)
llama_print_timings: prompt eval time =     183.61 ms /     7 tokens (   26.23 ms per token,    38.12 tokens per second)
llama_print_timings:        eval time =   17824.32 ms /   199 runs   (   89.57 ms per token,    11.16 tokens per second)
llama_print_timings:       total time =   18521.29 ms /   206 tokens


In [7]:
db = ChromaDataBase()
db.collection.count()

76095

In [36]:
def answer_on(question, verbose=True):
    result = db.query(
        query_texts=[question],
        n_results=10,
        where={'class': 1}
    )
    
    context = ' '.join(result['documents'][0])
    prompt = f'Here is the context: {context}. Here is the question: {question}. Here is the answer to this question:'
    
    if verbose:
        print(f'Prompt: {prompt}')
    
    res = llm(prompt, **generation_kwargs)
    return res["choices"][0]["text"]

In [49]:
questions = [
    'Where is the Catalan pro-independence party?',
    'Who was the president of US in 2018?',
    'Who was the president of US in 2016?',
    'How much forest is there in Slovenia?',
    'Is Donald Trump Republican Party nominee?',
    'Is Donald Trump Democratic Party nominee?',
    'What does the new U.S. tax code target?',
    'Who is Peru’s President in 2017?',
    'Did Google and Apple joined court papers?',
    'Was Silvio Berlusconi an Italian prime minister?'
]
answers = [
    'Catalonia, Spain',
    'Donald Trump',
    'Barack Obama',
    'About 60 percent of Slovenia is covered by forests',
    'Yes',
    'No',
    'The new U.S. tax code targets high-tax states',
    'Pedro Pablo Kuczynski',
    'Yes',
    'Yes'
]

for i in range(len(questions)):
    output = answer_on(questions[i], verbose=True)
    print(f'\nQuestion #{i}:\n{questions[i]}\nPossible answer: {answers[i]}\nLLM\'s answer: {output}\n')

Prompt: Here is the context: Earlier , another Catalan pro -independence party , Esquerra Republicana de Catalunya ERC , also said it would participate in the elections . On Saturday , Puigdemont - who PDeCAT said on Sunday would lead the party in the election - called for a united Catalan political front for independence from Spain and against the detention of his former members of government . MADRID Reuters - Catalan pro -independence party PdeCat Catalan Democratic Party said on Monday it will take part in a snap regional elections on Dec . 21 , a tacit admission that Catalonia s previous ruling party had been fired and parliament dissolved . But the majority of Catalans do want the opportunity to vote on whether to split from Spain . Knowing that in the end there won t be independence , I feel sorry for all the people tricked into thinking there could be and the divisions they ve driven through Catalan society . If he says he did not declare it , then the far -left Catalan party C

Llama.generate: prefix-match hit



Question #0:
Where is the Catalan pro-independence party?
Possible answer: Catalonia, Spain
LLM's answer:  The Catalan pro-independence party, PdeCat, is based in Catalonia, Spain.




llama_print_timings:        load time =     183.69 ms
llama_print_timings:      sample time =       3.40 ms /    22 runs   (    0.15 ms per token,  6476.30 tokens per second)
llama_print_timings: prompt eval time =   10365.30 ms /   413 tokens (   25.10 ms per token,    39.84 tokens per second)
llama_print_timings:        eval time =    1935.37 ms /    21 runs   (   92.16 ms per token,    10.85 tokens per second)
llama_print_timings:       total time =   12350.64 ms /   434 tokens


Prompt: Here is the context: President -elect Donald Trump , who takes office on Jan . Trump , who took office on Jan . Trump , who took office on Jan . Trump , a Republican elected on Nov . Trump , the front -runner to win the Republican nomination for the Nov . Trump , who is set to take office on Jan . Trump , who will take office on Jan . Trump , who takes office on Jan . Trump , who takes office on Jan . Trump , who takes office on Jan .. Here is the question: Who was the president of US in 2018?. Here is the answer to this question:


...

Llama.generate: prefix-match hit

llama_print_timings:        load time =     183.69 ms
llama_print_timings:      sample time =       1.89 ms /    12 runs   (    0.16 ms per token,  6359.30 tokens per second)
llama_print_timings: prompt eval time =   11252.84 ms /   447 tokens (   25.17 ms per token,    39.72 tokens per second)
llama_print_timings:        eval time =    1016.78 ms /    11 runs   (   92.44 ms per token,    10.82 tokens per second)
llama_print_timings:       total time =   12297.57 ms /   458 tokens



Question #8:
Did Google and Apple joined court papers?
Possible answer: Yes
LLM's answer:  Yes, both Google and Apple have joined court papers.

Prompt: Here is the context: He was Slovenia s prime minister from 2008 to 2012 , the years that led to the worst financial crisis in Slovenia s history . Benediktsson will become finance minister in the new government . Ablyazov , once a wealthy businessman who became a government minister , led an opposition political movement in the early 2000s until being convicted on corruption charges in 2002 . What other bank CEO is getting as much one -on-one face time with the prime minister ? As the prime minister made clear yesterday . . . Modi also named a new minister for the railways to restore confidence in the world s fourth -biggest rail network after a string of accidents , as well as a new leader to head a planned clean up of the river Ganges that has failed to make headway . This story corrects name of Australian prime minister in paragrap

Llama.generate: prefix-match hit



Question #9:
Was Silvio Berlusconi an Italian prime minister?
Possible answer: Yes
LLM's answer:  Yes, Silvio Berlusconi was an Italian prime minister. He served as Italy s prime minister on three separate occasions from 1994 to 2011.




llama_print_timings:        load time =     183.69 ms
llama_print_timings:      sample time =       6.23 ms /    41 runs   (    0.15 ms per token,  6577.89 tokens per second)
llama_print_timings: prompt eval time =    8246.08 ms /   331 tokens (   24.91 ms per token,    40.14 tokens per second)
llama_print_timings:        eval time =    3661.75 ms /    40 runs   (   91.54 ms per token,    10.92 tokens per second)
llama_print_timings:       total time =   12005.82 ms /   371 tokens
